# Representation Learning at the Houses of the Oireachtas in Ireland




Notebook dedicated to run representation learning on the Houses of the Oireachtas' data in Ireland.

This work is based on a Senator Representations' work of the US Congress by Nathaniel Tucker:

https://github.com/knathanieltucker/tf-keras-tutorial/blob/master/SenatorRepresentations.ipynb

The Oireachtas is the legislature of the Republic of Ireland.

The Oireachtas consists of:
- The President of Ireland
- The two houses of the Oireachtas:
    - Dáil Éireann (lower house)
    - Seanad Éireann (upper house)
    
Further info: https://en.wikipedia.org/wiki/Oireachtas

Information regarding the Houses of the Oireachtas is featured on their website, including legislation, and is the copyright of the Houses of the Oireachtas: https://beta.oireachtas.ie/. 

Another interesting resource is https://www.kildarestreet.com/ which is a searchable archive of everything that's been said in the Dáil and all written parliamentary questions since January 2004, everything in the Seanad since September 2002, and all Committee meetings since September 2012.

Open Data from the Houses of the Oireachtas can be accessed via:
https://beta.oireachtas.ie/en/open-data/

This includes a link to the open data APIs via a Swagger UI:
https://api.oireachtas.ie/

A vote in the Houses of the Oireachtas is also called a division. We will be looking at this divisions (votings) table for the house and the Seanad (Senate):

In [1]:
import requests # http://docs.python-requests.org/
from BeautifulSoup import BeautifulSoup
import json
import time
import csv
import numpy as np

In [2]:
def get_votings(chamber = 'seanad'):  # chamber: 'seanad' or 'dail' if chamber_type: 'house'
    
    votings_data = []

    table = "divisions" # votings table
    url = 'https://api.oireachtas.ie/v1/{}'.format(table)
    date_start = '1900-01-01'
    date_end = '2017-12-31'
    chamber_type = 'house' # 'committee' or 'house'

    batch = 500
    skip = 0
    limit = batch
    count = 1 # start

    while count > 0:

        params = dict(chamber_type=chamber_type,
                      chamber=chamber, 
                      date_start=date_start, 
                      date_end=date_end,
                      skip=skip, 
                      limit=limit)
        r = requests.get(url, params=params)

        print r.url
        # print "Status Code:", r.status_code
        # print "Headers:", r.headers

        # Add data from this batch
        contents = r.json()

        if 'message' in contents and contents['message'] == 'server error':
            print 'ERROR RETRIEVING DATA'
            return votings_data

        # Results    
        votings = contents['results'][:batch] # in order not to have duplicates
        votings_data.extend(votings)

        # Update count, more to retrieve?
        count = len(votings)
        print 'Number retrieved', count
        # Update skip and limit for query
        skip += batch
        limit += batch

        time.sleep(1)
        
    return votings_data

### Seanad Éireann (upper house)

In [3]:
# seanad_data = get_votings()

In [4]:
# with open('data/congress/Ireland/seanad_data.json', 'w') as outfile:
#      json.dump(seanad_data, outfile)

In [5]:
seanad_data = json.load(open('data/congress/Ireland/seanad_data.json'))

How many votings in the Senate?

In [7]:
len(seanad_data)

3530

In [8]:
# seanad_data[0]

### Dáil Éireann (lower house)

In [9]:
# dail_data = get_votings(chamber = 'dail')

In [10]:
# with open('data/congress/Ireland/dail_data.json', 'w') as outfile:
#      json.dump(dail_data, outfile)

In [11]:
dail_data = json.load(open('data/congress/Ireland/dail_data.json'))

In [12]:
len(dail_data)

5000

**Note**

The division count for the Dail is actually 7,329 but the API does only let me retrieve the first 5,000 votings

In [13]:
# dail_data[0]

Verify there is no duplicates:

In [14]:
def verify_no_voting_duplicates(vote_data):
    votes = []
    duplicates = []
    for vote in vote_data:
        uri = vote["division"]["uri"]
        if uri not in votes:
            # print uri
            votes.append(uri)
        else:
            duplicates.append(uri)
    return len(duplicates)

In [15]:
verify_no_voting_duplicates(seanad_data)

0

In [16]:
verify_no_voting_duplicates(dail_data)

0

Get members from the votings:

In [17]:
def get_members(vote_data):
    members = []
    for vote in vote_data:
        div = vote["division"]
        for outcome, voting in div["tallies"].iteritems():            
            if voting is not None:
                # print voting["showAs"] # Tá (YES), Níl (NO), Staon (Abstention)
                # print len(voting["members"])
                for member in voting["members"]:
                    member = member["member"]
                    #last_name, first_name = member["showAs"].split(",")
                    #first_name = first_name[:-1].strip()
                    #last_name = last_name.strip()
                    member_code = member["memberCode"]
                    # print member_code
                    if member_code is not None:
                        members.append(member_code)
    return members

**Senators**

In [18]:
senators = get_members(seanad_data)

In [19]:
senators[3]

u'Paudie-Coffey.S.2007-07-23'

In [20]:
len(set(senators))

197

**Deputies**

In [21]:
deputies = get_members(dail_data)

In [22]:
deputies[2]

u'Thomas-P-Broughan.D.1992-12-14'

In [23]:
len(set(deputies))

372

All members:

In [24]:
all_members = list(senators) # copy list
all_members.extend(deputies)

In [25]:
len(set(all_members))

478

In [26]:
for v, k in enumerate(set(all_members)):
    print v, k

0 Cecilia-Keaveney.D.1996-04-02
1 Maria-Byrne.S.2016-04-25
2 Helen-McEntee.D.2013-03-27
3 Martin-Kenny.D.2016-10-03
4 Eoghan-Murphy.D.2011-03-09
5 Willie-Penrose.D.1992-12-14
6 Fergal-Browne.S.2002-09-12
7 Colette-Kelleher.S.2016-04-25
8 Paudge-Connolly.D.2002-06-06
9 Neale-Richmond.S.2016-04-25
10 Joe-Behan.D.2007-06-14
11 David-Cullinane.S.2011-05-25
12 Cyprian-Brady.S.2002-09-12
13 Jackie-Cahill.D.2016-10-03
14 Eddie-Bohan.S.1987-04-25
15 Eamon-Scanlon.S.2002-09-12
16 Peter-Callanan.S.1997-09-17
17 James-Carroll.S.2009-11-26
18 Darragh-O'Brien.D.2007-06-14
19 Fidelma-Healy-Eames.S.2007-07-23
20 Jennifer-Murnane-O'Connor.S.2016-04-25
21 Kate-O'Connell.D.2016-10-03
22 Willie-O'Dea.D.1982-03-09
23 Fintan-Warfield.S.2016-04-25
24 Jerry-Buttimer.S.2007-07-23
25 Kate-Walsh.S.2002-09-12
26 Dessie-Ellis.D.2011-03-09
27 David-Stanton.D.1997-06-26
28 John-Browne.D.1982-12-14
29 Donnchadh-Ó-Laoghaire.D.2016-10-03
30 Shane-McEntee.D.2005-03-11
31 Eugene-Murphy.D.2016-10-03
32 Síle-de-Valera.D.1

In [27]:
# leave the first two blank for padding and not a member
member_to_id = { k: v + 2 for v, k in enumerate(set(all_members)) }

In [28]:
member_to_id

{u'Aengus-\xd3-Snodaigh.D.2002-06-06': 354,
 u'Aidan-Davitt.S.2016-04-25': 162,
 u'Aideen-Hayden.S.2011-05-25': 445,
 u'Aindrias-Moynihan.D.2016-10-03': 478,
 u'Alan-Farrell.D.2011-03-09': 38,
 u'Alan-Kelly.S.2007-07-23': 166,
 u'Alan-Shatter.D.1981-06-30': 54,
 u'Alex-White.S.2007-07-23': 371,
 u'Alice-Mary-Higgins.S.2016-04-25': 262,
 u'Andrew-Doyle.D.2007-06-14': 231,
 u'Ann-Ormonde.S.1993-02-17': 312,
 u'Ann-Phelan.D.2011-03-09': 395,
 u'Anne-Ferris.D.2011-03-09': 455,
 u'Anne-Rabbitte.D.2016-10-03': 448,
 u'Anthony-Lawlor.D.2011-03-09': 428,
 u'Aodh\xe1n-\xd3-R\xedord\xe1in.D.2011-03-09': 138,
 u'Arthur-Morgan.D.2002-06-06': 430,
 u'Arthur-Spring.D.2011-03-09': 113,
 u'Averil-Power.S.2011-05-25': 49,
 u'Barry-Andrews.D.2002-06-06': 391,
 u'Barry-Cowen.D.2011-03-09': 281,
 u"Batt-O'Keeffe.D.1987-03-10": 315,
 u'Bernard-Allen.D.1981-06-30': 91,
 u'Bernard-Durkan.D.1981-06-30': 475,
 u'Bertie-Ahern.D.1977-07-05': 317,
 u'Beverley-Flynn.D.1997-06-26': 139,
 u'Billy-Kelleher.S.1993-02-

In [29]:
senator_to_id = { k: v + 2 for v, k in enumerate(set(senators)) }
deputy_to_id = { k: v + 2 for v, k in enumerate(set(deputies)) }

In [30]:
len(senator_to_id), len(deputy_to_id)

(197, 372)

In [31]:
def get_member_id(name, collection):
    name = name.replace(" ", "-").replace(".", "")
    sens = [k for k, v in collection.iteritems() if k is not None and k.startswith(name)]
    if len(sens) > 0:
        return collection[sens[0]]
    return 1

In [32]:
get_member_id('Aideen Hayden', member_to_id)

445

In [33]:
get_member_id('Aideen Hayden', senator_to_id), get_member_id('Aideen Hayden', deputy_to_id)

(133, 1)

In [34]:
def get_member_party(name):
    page = requests.get('https://beta.oireachtas.ie/en/members/member/%s/' % (name)).text
    # dom = web.Element(page)
    # party = dom.content.split('<p class="bio-text">Party:</p> <p>')[1].split('</p')[0].strip()
    bs = BeautifulSoup(page)
    party = bs.find(text="Party:").findNext('p').contents[0].strip()
    return party

In [35]:
all_members[0], get_member_party(all_members[0])

(u'Victor-Boyhan.S.2016-04-25', u'Independent')

In [36]:
all_members[17], get_member_party(all_members[17])

(u'Brian-\xd3-Domhnaill.S.2007-07-23', u'Fianna F\xe1il')

In [37]:
parties = {}
for member in set(all_members):
    parties[member] = get_member_party(member)

In [38]:
parties

{u'Aengus-\xd3-Snodaigh.D.2002-06-06': u'Sinn F\xe9in',
 u'Aidan-Davitt.S.2016-04-25': u'Fianna F\xe1il',
 u'Aideen-Hayden.S.2011-05-25': u'Labour Party',
 u'Aindrias-Moynihan.D.2016-10-03': u'Fianna F\xe1il',
 u'Alan-Farrell.D.2011-03-09': u'Fine Gael',
 u'Alan-Kelly.S.2007-07-23': u'Labour',
 u'Alan-Shatter.D.1981-06-30': u'Fine Gael',
 u'Alex-White.S.2007-07-23': u'Labour Party',
 u'Alice-Mary-Higgins.S.2016-04-25': u'Independent',
 u'Andrew-Doyle.D.2007-06-14': u'Fine Gael',
 u'Ann-Ormonde.S.1993-02-17': u'Fianna F\xe1il',
 u'Ann-Phelan.D.2011-03-09': u'Labour Party',
 u'Anne-Ferris.D.2011-03-09': u'Labour Party',
 u'Anne-Rabbitte.D.2016-10-03': u'Fianna F\xe1il',
 u'Anthony-Lawlor.D.2011-03-09': u'Fine Gael',
 u'Aodh\xe1n-\xd3-R\xedord\xe1in.D.2011-03-09': u'Labour',
 u'Arthur-Morgan.D.2002-06-06': u'Sinn F\xe9in',
 u'Arthur-Spring.D.2011-03-09': u'Labour Party',
 u'Averil-Power.S.2011-05-25': u'Fianna F\xe1il',
 u'Barry-Andrews.D.2002-06-06': u'Fianna F\xe1il',
 u'Barry-Cowen.D.2

In [39]:
set(val for val in parties.values())

{u'Anti-Austerity Alliance - People Before Profit',
 u'Ceann Comhairle',
 u'Fianna F\xe1il',
 u'Fine Gael',
 u'Green Party',
 u'Independent',
 u'Independents 4 Change',
 u'Labour',
 u'Labour Party',
 u'Progressive Democrats',
 u'Renua',
 u'Sinn F\xe9in',
 u'Social Democrats',
 u'Socialist Party',
 u'Solidarity - People Before Profit'}

In [40]:
from collections import Counter
Counter([v for v in parties.values()])

Counter({u'Anti-Austerity Alliance - People Before Profit': 2,
         u'Ceann Comhairle': 1,
         u'Fianna F\xe1il': 165,
         u'Fine Gael': 122,
         u'Green Party': 12,
         u'Independent': 64,
         u'Independents 4 Change': 3,
         u'Labour': 12,
         u'Labour Party': 45,
         u'Progressive Democrats': 9,
         u'Renua': 3,
         u'Sinn F\xe9in': 33,
         u'Social Democrats': 2,
         u'Socialist Party': 1,
         u'Solidarity - People Before Profit': 4})

**Extracting sponsors and members against a bill**

The following function encapsulates the extract of sponsors and members against a bill based on a line of text manually inputed and in several formats:

In [41]:
# Sponsors and members against a Bill are specified in a text line where we will extract:
# 2 members that sponsored the bill
# 2 members that opposed to the bill
def get_sponsors(tellers_data, collection):
    
    #print tellers_data
    
    sp_first, sp_second, ag_first, ag_second = None, None, None, None
    
    if ";" in tellers_data:
        
        tellers = tellers_data.split(";")
        
        for teller in tellers:
            
            if teller.strip() == "":
                continue
            if "Tellers:" in teller:
                teller = teller.split("Tellers:")[1].strip()
            elif ":" in teller:
                teller = teller.split(":")[1].strip()

            if "," in teller:
                values = teller.split(",")
            elif ":" in teller:
                values = teller.split(":")
            else:
                #print "Bad format"
                continue
            if len(values) != 2:
                #print "BAD format"
                continue
                
            support, senators = values

            if "Senators" in senators:
                senators = senators.split("Senators")
                senators = senators[-1]
                
            if " and" in senators:
                senators = senators.split(" and")
            elif "and " in senators:
                senators = senators.split("and ")
            elif "an d":
                senators = senators.split("an d")
            else:
                senators = senators.split("and")
            
            if len(senators) > 1:
                first_senator, second_senator = senators[:2]
                second_senator = second_senator.strip()
            else:
                first_senator = senators[0]
                first_senator = first_senator.strip()
                second_senator = "None"
            
            support = support.strip()

            if support.encode('utf-8') == str('T\xc3\xa1'):
                sp_first = get_member_id(first_senator, collection)
                sp_second = get_member_id(second_senator, collection)
                # print "Sponsors", first_senator, sp_first, second_senator, sp_second
            elif support.encode('utf-8') == str('N\xc3\xadl'):
                ag_first = get_member_id(first_senator, collection)
                ag_second = get_member_id(second_senator, collection)
                # print "Against", first_senator, ag_first, second_senator, ag_second
                
    return sp_first, sp_second, ag_first, ag_second 

In [42]:
# Not all members will appear on the votings 
member_id_to_displayname = {}
senator_id_to_displayname = {}
deputy_id_to_displayname = {}
for member_code in set(all_members):
    member_id = member_to_id[member_code]
    name = member_code
    party = parties[name]
    if name is not None and '.' in name:
        name = name.split('.')[0].split('-')[-1] # last name
    if party is not None:
        name = name + ", " + party
    member_id_to_displayname[member_id] = name
    if member_code in senators:
        senator_id = senator_to_id[member_code]
        senator_id_to_displayname[senator_id] = name
    if member_code in deputies:
        deputy_id = deputy_to_id[member_code]
        deputy_id_to_displayname[deputy_id] = name

In [43]:
member_id_to_displayname

{2: u'Keaveney, Fianna F\xe1il',
 3: u'Byrne, Fine Gael',
 4: u'McEntee, Fine Gael',
 5: u'Kenny, Sinn F\xe9in',
 6: u'Murphy, Fine Gael',
 7: u'Penrose, Labour',
 8: u'Browne, Fine Gael',
 9: u'Kelleher, Independent',
 10: u'Connolly, Independent',
 11: u'Richmond, Fine Gael',
 12: u'Behan, Independent',
 13: u'Cullinane, Sinn F\xe9in',
 14: u'Brady, Fianna F\xe1il',
 15: u'Cahill, Fianna F\xe1il',
 16: u'Bohan, Fianna F\xe1il',
 17: u'Scanlon, Fianna F\xe1il',
 18: u'Callanan, Fianna F\xe1il',
 19: u'Carroll, Fianna F\xe1il',
 20: u"O'Brien, Fianna F\xe1il",
 21: u'Eames, Fine Gael',
 22: u"O'Connor, Fianna F\xe1il",
 23: u"O'Connell, Fine Gael",
 24: u"O'Dea, Fianna F\xe1il",
 25: u'Warfield, Sinn F\xe9in',
 26: u'Buttimer, Fine Gael',
 27: u'Walsh, Progressive Democrats',
 28: u'Ellis, Sinn F\xe9in',
 29: u'Stanton, Fine Gael',
 30: u'Browne, Fianna F\xe1il',
 31: u'Laoghaire, Sinn F\xe9in',
 32: u'McEntee, Fine Gael',
 33: u'Murphy, Fianna F\xe1il',
 34: u'Valera, Fianna F\xe1il',

In [44]:
# senator_id_to_displayname

In [45]:
# deputy_id_to_displayname

In [46]:
len(member_id_to_displayname), len(senator_id_to_displayname), len(deputy_id_to_displayname)

(478, 197, 372)

In [47]:
def get_vote_data(data, collection_to_ids, display_names):

    votings = []
    added, not_added = 0, 0

    for vote in data:

        div = vote["division"]
        #outcome = div["outcome"] # Carried, Lost, _
        #print div["date"]
        #print div["isBill"]
        #print div["category"]
        #print div["chamber"]["showAs"]
        #print div["debate"]["showAs"]

        # Bill sponsors
        sponsors = get_sponsors(div["tellers"], collection_to_ids)

        if None not in sponsors:

            sponsor, cosponsor, against, coagainst = sponsors

            tallies = div["tallies"]

            if tallies["taVotes"] is not None:
                for member in tallies["taVotes"]["members"]:
                    member = member["member"]
                    name = member["showAs"]
                    last_name = name.split(',')[0]
                    #uri = member["uri"]
                    #short = member["memberCode"]
                    if member["memberCode"] is not None:
                        member_id = collection_to_ids[member["memberCode"]]
                        party = parties[member["memberCode"]]
                        display_names[member_id] = last_name + ", " + party # replace names with utf8 encoding names
                        votings.append((1, member_id, sponsor, [cosponsor, against, coagainst])) 

            if tallies["nilVotes"] is not None:
                
                for member in tallies["nilVotes"]["members"]:
                    member = member["member"]
                    name = member["showAs"]
                    last_name = name.split(',')[0]
                    if member["memberCode"] is not None:
                        member_id = collection_to_ids[member["memberCode"]]
                        party = parties[member["memberCode"]]
                        display_names[member_id] = last_name + ", " + party
                        votings.append((0, member_id, sponsor, [cosponsor, against, coagainst]))   

            added += 1

        else: # Votings that could not be added

            not_added += 1
            
    return votings, added, not_added

In [48]:
seanad_votings, added, not_added = get_vote_data(seanad_data, senator_to_id, senator_id_to_displayname)

In [49]:
len(seanad_votings), added, not_added

(75487, 2802, 728)

In [50]:
seanad_votings

[(1, 100, 1, [117, 1, 90]),
 (1, 6, 1, [117, 1, 90]),
 (1, 122, 1, [117, 1, 90]),
 (1, 87, 1, [117, 1, 90]),
 (1, 196, 1, [117, 1, 90]),
 (1, 146, 1, [117, 1, 90]),
 (1, 55, 1, [117, 1, 90]),
 (1, 60, 1, [117, 1, 90]),
 (1, 25, 1, [117, 1, 90]),
 (1, 91, 1, [117, 1, 90]),
 (1, 195, 1, [117, 1, 90]),
 (1, 129, 1, [117, 1, 90]),
 (1, 190, 1, [117, 1, 90]),
 (1, 81, 1, [117, 1, 90]),
 (1, 39, 1, [117, 1, 90]),
 (1, 114, 1, [117, 1, 90]),
 (1, 72, 1, [117, 1, 90]),
 (1, 148, 1, [117, 1, 90]),
 (1, 117, 1, [117, 1, 90]),
 (1, 99, 1, [117, 1, 90]),
 (1, 79, 1, [117, 1, 90]),
 (1, 193, 1, [117, 1, 90]),
 (1, 21, 1, [117, 1, 90]),
 (1, 73, 1, [117, 1, 90]),
 (0, 53, 1, [117, 1, 90]),
 (0, 157, 1, [117, 1, 90]),
 (0, 104, 1, [117, 1, 90]),
 (0, 124, 1, [117, 1, 90]),
 (0, 90, 1, [117, 1, 90]),
 (0, 33, 1, [117, 1, 90]),
 (0, 45, 1, [117, 1, 90]),
 (1, 53, 1, [101, 1, 117]),
 (1, 157, 1, [101, 1, 117]),
 (1, 172, 1, [101, 1, 117]),
 (1, 124, 1, [101, 1, 117]),
 (1, 90, 1, [101, 1, 117]),
 (1, 15

In [51]:
dail_votings, added, not_added = get_vote_data(dail_data, deputy_to_id, deputy_id_to_displayname)

In [52]:
len(dail_votings), added, not_added

(148744, 3574, 1426)

In [53]:
dail_votings

[(1, 350, 1, [332, 1, 62]),
 (1, 174, 1, [332, 1, 62]),
 (1, 191, 1, [332, 1, 62]),
 (1, 262, 1, [332, 1, 62]),
 (1, 136, 1, [332, 1, 62]),
 (1, 353, 1, [332, 1, 62]),
 (1, 298, 1, [332, 1, 62]),
 (1, 68, 1, [332, 1, 62]),
 (1, 70, 1, [332, 1, 62]),
 (1, 235, 1, [332, 1, 62]),
 (1, 19, 1, [332, 1, 62]),
 (1, 253, 1, [332, 1, 62]),
 (1, 54, 1, [332, 1, 62]),
 (1, 326, 1, [332, 1, 62]),
 (1, 170, 1, [332, 1, 62]),
 (1, 4, 1, [332, 1, 62]),
 (1, 213, 1, [332, 1, 62]),
 (1, 285, 1, [332, 1, 62]),
 (1, 256, 1, [332, 1, 62]),
 (1, 332, 1, [332, 1, 62]),
 (1, 267, 1, [332, 1, 62]),
 (1, 208, 1, [332, 1, 62]),
 (1, 331, 1, [332, 1, 62]),
 (1, 149, 1, [332, 1, 62]),
 (1, 240, 1, [332, 1, 62]),
 (1, 23, 1, [332, 1, 62]),
 (1, 272, 1, [332, 1, 62]),
 (1, 209, 1, [332, 1, 62]),
 (1, 310, 1, [332, 1, 62]),
 (1, 367, 1, [332, 1, 62]),
 (1, 49, 1, [332, 1, 62]),
 (1, 194, 1, [332, 1, 62]),
 (1, 220, 1, [332, 1, 62]),
 (1, 363, 1, [332, 1, 62]),
 (1, 107, 1, [332, 1, 62]),
 (1, 336, 1, [332, 1, 62]),


In [54]:
len(seanad_votings), len(dail_votings)

(75487, 148744)

### Senate votings

~75 examples of (vote, senator voting, sponsor, cosponsor, against, coagainst) tuples is pretty good (we could of course scrape more).

In [55]:
y = [d[0] for d in seanad_votings]

In [56]:
# len(y)
Counter(y)

Counter({0: 36729, 1: 38758})

In [57]:
# again we pad
def pad_or_crop(lst, l=10):
    return (lst + [0] * l)[:10]

In [58]:
x_1 = np.array(map(lambda x: x[1], seanad_votings))
x_2 = np.array(map(lambda x: x[2], seanad_votings))
x_3 = np.array(map(lambda x: pad_or_crop(x[3]), seanad_votings))
x = [x_1, x_2, x_3]

In [59]:
# x

In [60]:
# len(x)

In [61]:
# we add in padding and unknown senators
senator_id_to_displayname[0] = '<PAD>'
senator_id_to_displayname[1] = '<NOT A SENATOR>'

In [62]:
# senator_id_to_displayname

In [63]:
# this gives us how many representations:
len(senator_id_to_displayname)

199

In [64]:
# we again need to write down the metadata
with open('data/congress/Ireland/senator_metadata.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, value in sorted(senator_id_to_displayname.items()):
        writer.writerow([value.encode('utf8')])

In [65]:
# finally we build our model
from keras.layers import concatenate
from keras.layers import Dense, Input, Flatten
from keras.layers import MaxPooling1D, Embedding

embedding_layer = Embedding(len(senator_id_to_displayname), 100)

# train a 1D convnet with global maxpooling
voting = voting_input = Input(shape=(1,), dtype='int32')
voting = embedding_layer(voting)
voting = Dense(32, activation='relu')(voting)
voting = Dense(32, activation='relu')(voting)

sponsor = sponsor_input = Input(shape=(1,), dtype='int32')
sponsor = embedding_layer(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)

cosponsor = cosponsor_input = Input(shape=(10,), dtype='int32')
cosponsor = embedding_layer(cosponsor)
cosponsor = MaxPooling1D(10)(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)

combined = concatenate([voting, sponsor, cosponsor])
combined = Dense(32, activation='relu')(combined)
combined = Dense(1, activation='sigmoid')(combined)

Using TensorFlow backend.


In [66]:
from keras.models import Model
from keras.callbacks import TensorBoard

model = Model([voting_input, sponsor_input, cosponsor_input], combined)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

embedding_metadata = {
    embedding_layer.name: '../senator_metadata.csv'
}

model.fit([x_1, x_2, x_3], np.array(y).reshape(-1, 1, 1),
          batch_size=128,
          epochs=10,
          validation_split=0.2,
          callbacks=[TensorBoard(log_dir='data/congress/Ireland/senator_reps', 
                                 embeddings_freq=1,
                                 embeddings_metadata=embedding_metadata)])

Train on 60389 samples, validate on 15098 samples
Epoch 1/10
60389/60389 [==============================] - 4s 66us/step - loss: 0.6901 - acc: 0.5293 - val_loss: 0.6955 - val_acc: 0.5084
Epoch 2/10
60389/60389 [==============================] - 3s 56us/step - loss: 0.6608 - acc: 0.5884 - val_loss: 0.7197 - val_acc: 0.5395
Epoch 3/10
60389/60389 [==============================] - 3s 56us/step - loss: 0.6377 - acc: 0.6111 - val_loss: 0.7512 - val_acc: 0.5455
Epoch 4/10
60389/60389 [==============================] - 4s 63us/step - loss: 0.6304 - acc: 0.6112 - val_loss: 0.7898 - val_acc: 0.5201
Epoch 5/10
60389/60389 [==============================] - 3s 57us/step - loss: 0.6268 - acc: 0.6135 - val_loss: 0.7975 - val_acc: 0.5401
Epoch 6/10
60389/60389 [==============================] - 3s 57us/step - loss: 0.6245 - acc: 0.6130 - val_loss: 0.8171 - val_acc: 0.5131
Epoch 7/10
60389/60389 [==============================] - 4s 65us/step - loss: 0.6227 - acc: 0.6145 - val_loss: 0.8245 - val_acc

In data/congress/Ireland, launch TensorBoard:
    
> davids-air:Ireland dazconap$ tensorboard --logdir=senator_reps/

Go to TensorBoard:

> http://localhost:6006/#projector
        
In TensorBoard, we can look at the representations created in our model using t-SNE or PCA. A t-SNE analysis using more than a 1.1K iterations divides the senators in different groups.

### Dail votings

In [67]:
y = [d[0] for d in dail_votings]

In [68]:
# len(y)
Counter(y)

Counter({0: 69948, 1: 78796})

In [69]:
x_1 = np.array(map(lambda x: x[1], dail_votings))
x_2 = np.array(map(lambda x: x[2], dail_votings))
x_3 = np.array(map(lambda x: pad_or_crop(x[3]), dail_votings))
x = [x_1, x_2, x_3]

In [70]:
# we add in padding and unknown senators
deputy_id_to_displayname[0] = '<PAD>'
deputy_id_to_displayname[1] = '<NOT A DEPUTY>'

In [71]:
# deputy_id_to_displayname

In [72]:
len(deputy_id_to_displayname)

374

In [73]:
# we again need to write down the metadata
with open('data/congress/Ireland/deputy_metadata.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, value in sorted(deputy_id_to_displayname.items()):
        writer.writerow([value.encode('utf8')])

In [74]:
# finally we build our model
from keras.layers import concatenate
from keras.layers import Dense, Input, Flatten
from keras.layers import MaxPooling1D, Embedding

embedding_layer = Embedding(len(deputy_id_to_displayname), 100)

# train a 1D convnet with global maxpooling
voting = voting_input = Input(shape=(1,), dtype='int32')
voting = embedding_layer(voting)
voting = Dense(32, activation='relu')(voting)
voting = Dense(32, activation='relu')(voting)

sponsor = sponsor_input = Input(shape=(1,), dtype='int32')
sponsor = embedding_layer(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)

cosponsor = cosponsor_input = Input(shape=(10,), dtype='int32')
cosponsor = embedding_layer(cosponsor)
cosponsor = MaxPooling1D(10)(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)

combined = concatenate([voting, sponsor, cosponsor])
combined = Dense(32, activation='relu')(combined)
combined = Dense(1, activation='sigmoid')(combined)

In [75]:
from keras.models import Model
from keras.callbacks import TensorBoard

model = Model([voting_input, sponsor_input, cosponsor_input], combined)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

embedding_metadata = {
    embedding_layer.name: '../deputy_metadata.csv'
}

model.fit([x_1, x_2, x_3], np.array(y).reshape(-1, 1, 1),
          batch_size=128,
          epochs=10,
          validation_split=0.2,
          callbacks=[TensorBoard(log_dir='data/congress/Ireland/deputy_reps', 
                                 embeddings_freq=1,
                                 embeddings_metadata=embedding_metadata)])

Train on 118995 samples, validate on 29749 samples
Epoch 1/10
118995/118995 [==============================] - 8s 68us/step - loss: 0.5804 - acc: 0.6897 - val_loss: 0.5335 - val_acc: 0.7975
Epoch 2/10
118995/118995 [==============================] - 7s 58us/step - loss: 0.5218 - acc: 0.7450 - val_loss: 0.5299 - val_acc: 0.8003
Epoch 3/10
118995/118995 [==============================] - 7s 60us/step - loss: 0.5088 - acc: 0.7506 - val_loss: 0.5528 - val_acc: 0.7875
Epoch 4/10
118995/118995 [==============================] - 7s 60us/step - loss: 0.5013 - acc: 0.7574 - val_loss: 0.5215 - val_acc: 0.8036
Epoch 5/10
118995/118995 [==============================] - 7s 57us/step - loss: 0.4950 - acc: 0.7603 - val_loss: 0.5172 - val_acc: 0.8039
Epoch 6/10
118995/118995 [==============================] - 7s 62us/step - loss: 0.4876 - acc: 0.7640 - val_loss: 0.5075 - val_acc: 0.8036
Epoch 7/10
118995/118995 [==============================] - 7s 62us/step - loss: 0.4803 - acc: 0.7682 - val_loss: 0

### Utils

In [76]:
print '\xc3\xa1' == str('á')
print str('á').decode('utf-8').encode('utf-8')
print str('á').decode('utf-8').encode('utf-8') == str('\xc3\xa1')
print str('Tá').decode('utf-8').encode('utf-8') == str('T\xc3\xa1')
print str('Níl').decode('utf-8').encode('utf-8') == str('N\xc3\xadl')

True
á
True
True
True
